In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics  import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stop words in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
df = pd.read_csv('D:\PROJECTS FOR ANALISYS\ML projects\Fake News Prediction using Machine Learning/train.csv')

### Data Exploration 

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [7]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
df.shape

(20800, 5)

In [10]:
df.count() 

id        20800
title     20242
author    18843
text      20761
label     20800
dtype: int64

In [11]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
df.dtypes

id         int64
title     object
author    object
text      object
label      int64
dtype: object

In [13]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [14]:
df.author.unique()

array(['Darrell Lucus', 'Daniel J. Flynn', 'Consortiumnews.com', ...,
       'D. Samuelson', 'Judge Andrew Napolitano',
       'Michael J. de la Merced and Rachel Abrams'], dtype=object)

Data Processing

In [15]:
# missig data ?

for col in df.columns:
    prct_missing = np.mean(df[col].isnull()).round(3)
    print(f'{col} ->  {prct_missing}')

id ->  0.0
title ->  0.027
author ->  0.094
text ->  0.002
label ->  0.0


In [16]:
# replacing the null values with empty
df=df.fillna('')

In [36]:
df.dropna(inplace=True)
df.replace(np.nan,'',inplace=True)
df.drop_duplicates(inplace=True)

In [37]:
df_missing_values=df.isnull().sum()
df_missing_values

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [19]:
df_cells = np.product(df.shape)

In [20]:
the_percentage_of_missing = (df_missing_values/df_cells)*100
the_percentage_of_missing

id        0.0
title     0.0
author    0.0
text      0.0
label     0.0
dtype: float64

In [28]:
df['author'].value_counts()

                                             1957
Pam Key                                       243
admin                                         193
Jerome Hudson                                 166
Charlie Spiering                              141
                                             ... 
Jeremy R. Hammond                               1
Vic Bishop                                      1
Douglas Martin                                  1
Najim Rahim and Fahim Abed                      1
Michael J. de la Merced and Rachel Abrams       1
Name: author, Length: 4202, dtype: int64

In [32]:
df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [34]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [30]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus---House Dem Aide: We Didn’t Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn---FLYNN: Hillary Clinton, Big ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com---Why the Truth Might Get Y...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss---15 Civilians Killed In Singl...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy---Iranian woman jailed for fict...


Data Pre-Processing

In [21]:
# combining the author and title columns
df['content']= df['author']+'---'+df['title']

In [22]:
df['content']

0        Darrell Lucus---House Dem Aide: We Didn’t Even...
1        Daniel J. Flynn---FLYNN: Hillary Clinton, Big ...
2        Consortiumnews.com---Why the Truth Might Get Y...
3        Jessica Purkiss---15 Civilians Killed In Singl...
4        Howard Portnoy---Iranian woman jailed for fict...
                               ...                        
20795    Jerome Hudson---Rapper T.I.: Trump a ’Poster C...
20796    Benjamin Hoffman---N.F.L. Playoffs: Schedule, ...
20797    Michael J. de la Merced and Rachel Abrams---Ma...
20798    Alex Ansary---NATO, Russia To Hold Parallel Ex...
20799            David Swanson---What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

Data Pre-processing

In [39]:
x= df.drop(columns='label' ,axis = 1)
y=df['label']

In [41]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

### Stemming
reduce the word into it's roots 

In [42]:
port_stem = PorterStemmer()

In [43]:
def steeming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]  
    stemmed_content=stemmed_content=' '.join(stemmed_content)
    return(stemmed_content) 

In [44]:
df['content']=df['content'].apply(steeming)

In [45]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [47]:
x = df['content'].values
y = df['label'].values

In [48]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [49]:
print(y)

[1 0 1 ... 0 1 1]


In [50]:
y.shape

(20800,)

In [51]:
# convert the text into meaningful num to get the model 

vectorizing= TfidfVectorizer()
vectorizing.fit(x)

x=vectorizing.transform(x)


In [52]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

splitting data set to train and test


In [53]:
x_train, x_test,y_train, y_test= train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [54]:
print(x_train, y_train)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

### Training the model : Logistic Regression

In [55]:
model = LogisticRegression()

In [56]:
model.fit(x_train, y_train)

LogisticRegression()

Accuracy_score and prediction

In [63]:
x_train_prediction=model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)*100

In [64]:
print('accuracy of training data => ',training_data_accuracy.round(2))

accuracy of training data =>  98.66


In [65]:
x_test_prediction=model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)*100

In [69]:
print('accuracy of test data => ',test_data_accuracy.round(2))

accuracy of test data =>  97.91


Making a predictive system

In [77]:
x_new =x_test[5]

prediction=model.predict(x_new)
print(prediction)

if(prediction[0] == 0):
    print('the news is real')
else:
    print('the news is fake')

[1]
the news is fake


In [78]:
y_test[5]

1